# Export data to files with Suitcase

In this notebook you will:

* Export saved data to CSV and TIFF.
* See a preview of how exporting data live during a scan works.

## Configuration
Below, we will connect to EPICS IOC(s) controlling simulated hardware in lieu of actual motors, detectors. The IOCs should already be running in the background. Run this command to verify that they are running: it should produce output with RUNNING on each line. In the event of a problem, edit this command to replace `status` with `restart all` and run again.

In [1]:
!supervisorctl -c supervisor/supervisord.conf status

decay                            RUNNING   pid 4869, uptime 0:00:33
mini_beamline                    RUNNING   pid 4870, uptime 0:00:33
random_walk                      RUNNING   pid 4871, uptime 0:00:33
random_walk_horiz                RUNNING   pid 4872, uptime 0:00:33
random_walk_vert                 RUNNING   pid 4873, uptime 0:00:33
simple                           RUNNING   pid 4874, uptime 0:00:33
thermo_sim                       RUNNING   pid 4875, uptime 0:00:33
trigger_with_pc                  RUNNING   pid 4876, uptime 0:00:33


In [2]:
%run scripts/beamline_configuration.py

## Scalar data to CSV

In [3]:
RE(scan([ph], motor_ph, -1, 1, 5), sample_comp='Ni', barcode='12345')



Transient Scan ID: 3     Time: 2020-05-18 20:35:02
Persistent Unique Scan ID: 'e91b54fd-2c46-42f2-82ca-07712e63caba'
New stream: 'primary'


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

+-----------+------------+------------+------------+
|   seq_num |       time |   motor_ph |     ph_det |
+-----------+------------+------------+------------+
|         1 | 20:35:02.5 |     -1.000 |      93606 |


|         2 | 20:35:02.5 |     -0.500 |      95863 |
|         3 | 20:35:02.5 |      0.000 |      96823 |
|         4 | 20:35:02.5 |      0.500 |      97283 |
|         5 | 20:35:02.6 |      1.000 |      95514 |


+-----------+------------+------------+------------+
generator scan ['e91b54fd'] (scan num: 3)





('e91b54fd-2c46-42f2-82ca-07712e63caba',)

In [4]:
import suitcase.csv

In [5]:
suitcase.csv.export(db[-1].documents(fill=True), 'my_exported_files/')

{'stream_data': [PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/e91b54fd-2c46-42f2-82ca-07712e63caba-primary.csv')]}

In [6]:
suitcase.csv.export(db[-1].documents(fill=True), 'my_exported_files/', file_prefix='{start[sample_comp]}-{start[barcode]}-')

{'stream_data': [PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni-12345-primary.csv')]}

In [7]:
suitcase.csv.export(db[-1].documents(fill=True), 'my_exported_files/', file_prefix='{start[sample_comp]}/{start[barcode]}-')

{'stream_data': [PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345-primary.csv')]}

## Image data to TIFF

In [8]:
RE(scan([spot], mtr_spotx, -1, 1, 5), sample_comp='Ni', barcode='12345')



Transient Scan ID: 4     Time: 2020-05-18 20:35:02
Persistent Unique Scan ID: '6f025be2-ec32-4c89-920e-a4713ca9146f'


New stream: 'primary'


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

+-----------+------------+-------------+------------+
|   seq_num |       time | motor_spotx |   spot_roi |
+-----------+------------+-------------+------------+
|         1 | 20:35:03.0 |          -1 | 1088521106 |


|         2 | 20:35:03.2 |          -0 | 1100961537 |


|         3 | 20:35:03.3 |           0 | 1107708863 |
|         4 | 20:35:03.4 |           0 | 1113642989 |


|         5 | 20:35:03.5 |           1 | 1118526778 |


+-----------+------------+-------------+------------+
generator scan ['6f025be2'] (scan num: 4)





('6f025be2-ec32-4c89-920e-a4713ca9146f',)

### TIFF stack, a 3D block of images in one file

In [9]:
import suitcase.tiff_stack

In [10]:
suitcase.tiff_stack.export(db[-1].documents(fill=True), 'my_exported_files/', file_prefix='{start[sample_comp]}/{start[barcode]}-')

{'stream_data': [PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345-primary-spot_img.tiff')]}

### TIFF series

In [11]:
import suitcase.tiff_series

Unlike CSV or TIFF stack, this exporter creates separate file per row (Event document). We can include information from a specific Event in the filename. Here, we include the motor position.

In [12]:
suitcase.tiff_series.export(db[-1].documents(fill=True), 'my_exported_files/', file_prefix='{start[sample_comp]}/{start[barcode]}/{event[data][motor_spotx]}-')

{'stream_data': [PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345/-1.0-primary-spot_img-0.tiff'),
  PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345/-0.5-primary-spot_img-1.tiff'),
  PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345/0.0-primary-spot_img-2.tiff'),
  PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345/0.5-primary-spot_img-3.tiff'),
  PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345/1.0-primary-spot_img-4.tiff')]}

## "Beta" feature: Set up live exporting

*We plan to streamline this in the future, but here is a demonstration that it is possible.*

In [13]:
# Execute this once to configure RE to send data to an exporter.
# At the beamline, this would normally be done in a script automatically at startup.

from event_model import RunRouter
from databroker.core import discover_handlers

def factory(name, start_doc):

    serializer = suitcase.tiff_series.Serializer(
        'my_exported_files/', file_prefix='{start[sample_comp]}/{start[barcode]}-{start[uid]:.8}-')

    def cb(name, doc):
        serializer(name, doc)

    return [cb], []


rr = RunRouter([factory], discover_handlers())
RE.subscribe(rr)

2

In [14]:
RE(count([spot], num=30, delay=1), sample_comp='Lab6', barcode='12345')



Transient Scan ID: 5     Time: 2020-05-18 20:35:03
Persistent Unique Scan ID: '155c8507-dd55-4414-9d0e-23c6116b008c'
New stream: 'primary'


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

+-----------+------------+------------+
|   seq_num |       time |   spot_roi |
+-----------+------------+------------+
|         1 | 20:35:04.0 | 1130913909 |


|         2 | 20:35:04.9 | 1098529710 |


|         3 | 20:35:05.9 | 1053656005 |


...

|        28 | 20:35:31.0 | 1084613167 |


|        29 | 20:35:32.0 | 1131639869 |


|        30 | 20:35:33.0 | 1095412029 |


+-----------+------------+------------+
generator count ['155c8507'] (scan num: 5)





('155c8507-dd55-4414-9d0e-23c6116b008c',)